Claro, te explicaré el código paso a paso:

### Importación de Librerías



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

- **requests**: Para realizar solicitudes HTTP y obtener el contenido de las páginas web.
- **BeautifulSoup**: Para analizar y extraer datos del HTML.
- **pandas**: Para manipular y estructurar los datos en un DataFrame.

### Definición de la Función `scrape_books`



In [ ]:
def scrape_books(min_rating, max_price):

- **Parámetros**:
  - `min_rating`: La calificación mínima de los libros que queremos filtrar.
  - `max_price`: El precio máximo de los libros que queremos filtrar.

### Inicialización de Variables



In [ ]:
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    book_data = []

- **base_url**: La URL base del sitio web con un marcador de posición para el número de página.
- **book_data**: Una lista vacía para almacenar los datos de los libros.

### Bucle para Recorrer las Páginas



In [ ]:
    for page in range(1, 51):  # Assuming there are 50 pages
        url = base_url.format(page)
        response = requests.get(url)
        response.encoding = 'utf-8'  # Ensure correct encoding
        soup = BeautifulSoup(response.text, 'html.parser')

- **Bucle**: Itera sobre las páginas del sitio web (asumiendo que hay 50 páginas).
- **url**: Formatea la URL base con el número de página actual.
- **response**: Realiza una solicitud HTTP a la URL y obtiene la respuesta.
- **response.encoding**: Asegura que la codificación de la respuesta sea `utf-8`.
- **soup**: Analiza el contenido HTML de la respuesta usando BeautifulSoup.

### Extracción de Datos de los Libros



In [ ]:
        books = soup.select('article.product_pod')
        
        for book in books:
            rating = book.p['class'][1]
            rating = convert_rating(rating)
            price = book.select_one('p.price_color').text
            price = float(price.replace('£', '').replace('Â', '').strip())  # Remove unwanted characters

- **books**: Selecciona todos los elementos HTML que representan libros.
- **rating**: Extrae la clase CSS que contiene la calificación del libro y la convierte a un número usando `convert_rating`.
- **price**: Extrae el precio del libro, elimina caracteres no deseados (`£` y `Â`), y lo convierte a un valor flotante.

### Filtrado de Libros



In [ ]:
            if rating >= min_rating and price <= max_price:
                title = book.h3.a['title']
                book_url = "http://books.toscrape.com/catalogue/" + book.h3.a['href']
                book_response = requests.get(book_url)
                book_response.encoding = 'utf-8'  # Ensure correct encoding
                book_soup = BeautifulSoup(book_response.text, 'html.parser')

- **Condición**: Filtra los libros que cumplen con los criterios de `min_rating` y `max_price`.
- **title**: Extrae el título del libro.
- **book_url**: Construye la URL completa del libro.
- **book_response**: Realiza una solicitud HTTP a la URL del libro y obtiene la respuesta.
- **book_response.encoding**: Asegura que la codificación de la respuesta sea `utf-8`.
- **book_soup**: Analiza el contenido HTML de la respuesta del libro usando BeautifulSoup.

### Extracción de Detalles del Libro



In [ ]:
                upc = book_soup.select_one('table.table.table-striped tr:nth-child(1) td').text
                genre = book_soup.select_one('ul.breadcrumb li:nth-child(3) a').text
                availability = book_soup.select_one('p.instock.availability').text.strip()
                description = book_soup.select_one('#product_description ~ p')
                description = description.text if description else "No description available"

- **upc**: Extrae el UPC del libro.
- **genre**: Extrae el género del libro.
- **availability**: Extrae la disponibilidad del libro.
- **description**: Extrae la descripción del libro, si está disponible.

### Almacenamiento de Datos



In [ ]:
                book_data.append({
                    "UPC": upc,
                    "Title": title,
                    "Price (£)": price,
                    "Rating": rating,
                    "Genre": genre,
                    "Availability": availability,
                    "Description": description
                })

- **book_data.append**: Añade un diccionario con los detalles del libro a la lista `book_data`.

### Creación del DataFrame



In [ ]:
    return pd.DataFrame(book_data)

- **pd.DataFrame**: Convierte la lista de diccionarios `book_data` en un DataFrame de pandas y lo retorna.

### Función Auxiliar `convert_rating`



In [ ]:
def convert_rating(rating_str):
    rating_dict = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    return rating_dict.get(rating_str, 0)

- **convert_rating**: Convierte la calificación en formato de texto a un número entero usando un diccionario de mapeo.

### Uso del Ejemplo



In [ ]:
# Example usage
df = scrape_books(min_rating=4, max_price=20)
print(df)

- **df**: Llama a la función `scrape_books` con una calificación mínima de 4 y un precio máximo de 20, y almacena el resultado en `df`.
- **print(df)**: Imprime el DataFrame resultante.